In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from matplotlib import pyplot as plt
from keras.callbacks import TensorBoard
from keras.utils import to_categorical

In [ ]:
path_file_xlsx = 'PATH'
df = pd.read_excel(path_file_xlsx)

In [ ]:
df.columns

In [ ]:
X = df[['crimen', 'oxidonitroso', 'cuartos', 'distanciaempleo', 'movilidad', 'impuestos', 'tasaeducativa',
        'pobreza']]
#X = X.assign(bias=np.ones(len(X)))
y = df['valor']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
glm_gamma = sm.GLM(y_train, X_train, family=sm.families.Gamma())
glm_results = glm_gamma.fit()
print(glm_results.summary())

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mean_absolute_error(y, glm_results.predict(X))

In [ ]:
np.sqrt(mean_squared_error(y, glm_results.predict(X)))

In [ ]:
scaler = MinMaxScaler((-1,1))
scaler.fit(X)
X_train2 = scaler.transform(X_train)
X_test2 = scaler.transform(X_test)

In [ ]:
# define base model
def baseline_model(neurons=None):
    model = Sequential()
    model.add(Dense(65, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(40, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_absolute_error', optimizer='adam')
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
#estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=1000, batch_size=15, verbose=0)

In [ ]:
#for i in np.arange(10, 101, 5):
model = baseline_model()
i=50
tensorboard = TensorBoard(log_dir="logs/three_layer_without_rio_{}".format(i))
model.fit(X_train2, y_train.values, batch_size=50, epochs=3000, verbose=False, callbacks=[tensorboard], validation_data = (X_test2, y_test.values))
#df_test = df_test.assign(predict_keras=model.predict(X_test))

In [ ]:
np.sqrt(mean_squared_error(y_test, model.predict(X_test2)))

In [ ]:
mean_absolute_error(y_test, model.predict(X_test2))

In [ ]:
y_pred_dnn = model.predict(X_test2)
y_pred_dnn = y_pred_dnn.reshape(127)

In [ ]:
pd.DataFrame({'y_test': y_test, 'y_pred_glm': glm_results.predict(X_test), 'y_pred_dnn': y_pred_dnn}).head()